In [4]:
import streamlit_functions as sf
import importlib as imp
import geopandas as gpd
import matplotlib.pyplot as plt
import functions as f
# debugging
import importlib as imp
imp.reload(f)

# graph
import networkx as nx
import h3
from functions import H3_INDEX_RESOLUTION
from shapely.geometry import Point

### find closest bikelines based on the h3 index

In [5]:
bikelane_all = gpd.read_parquet('dataset/raw_unprocessed/bikelane_dk_WGS84_h3_indexed.parquet')

In [7]:
coordinates1 = [[11.581726, 55.606109], [11.699829, 55.592143]] # sjeland
coordinates2 = [[12.473907, 55.717571], [12.561798, 55.695132]] # in Copenhagen
coordinates3 = [[12.200317, 55.864524], [11.881714, 55.523967]] # big distance

In [147]:
# Assuming 'bikelane_all' is your GeoDataFrame of bike lanes
# and 'coordinates2' are your start and end points in Copenhagen

# Convert points to Shapely Points
start_point = Point(coordinates2[0])
end_point = Point(coordinates2[1])


# Function to find the closest LineString to a point, considering H3 index
def find_closest_objects(point, gdf):
    point_h3_index = f.get_h3_index_from_point(point)
    # Filter linestrings by H3 index
    filtered_gdf = gdf[gdf['h3_index'] == point_h3_index]
    if len(filtered_gdf) == 0:
        # If no lines are found in the same H3 index, search in the neighboring indexes
        neighbors = h3.k_ring(point_h3_index, 1)
        filtered_gdf = gdf[gdf['h3_index'].isin(neighbors)]
    
    min_dist = float('inf')
    closest_line = None
    for row in filtered_gdf.itertuples():
        # Calculate the distance from the point to the current line
        dist = point.distance(row.geometry)
        if dist < min_dist:
            min_dist = dist
            closest_line = row.geometry
            closest_edge = row.Index  # itertuples gives us the index in the 'Index' attribute
            closest_line_point = f.closest_coordinate_on_linestring(point, closest_line)
    return closest_line, closest_edge[0:2], closest_line_point, filtered_gdf

# Find closest LineStrings to start and end points

closest_line_start, closest_edge_start, closest_line_point_start, filtered_gdf_1 = find_closest_objects(start_point, bikelane_all)
closest_line_end, closest_edge_end, closest_line_point_end, filtered_gdf_2 = find_closest_objects(end_point, bikelane_all)
print(closest_line_start, closest_edge_start, closest_line_point_start)
print(closest_line_end, closest_edge_end, closest_line_point_end)


LINESTRING (12.4777437 55.7171138, 12.4768664 55.7174166, 12.4766548 55.7174636, 12.4764077 55.7174804, 12.4761584 55.7174662, 12.4751326 55.7172594, 12.4732513 55.7168712, 12.4730847 55.7168092, 12.4727463 55.716573, 12.4725479 55.7165042, 12.4715788 55.716314, 12.4698784 55.7159803, 12.4697847 55.7159517, 12.4694657 55.7157058, 12.4693875 55.7156994, 12.4692411 55.7156978) (283279508, 423158236) POINT (12.4732513 55.7168712)
LINESTRING (12.5614365 55.6958225, 12.5613083 55.6947673) (269598178, 7773050) POINT (12.5613083 55.6947673)


### Preparing for shortest path 

In [39]:
# Initialize an empty list to store edges
edge_list = []

# Loop through the DataFrame using itertuples
for row in bikelane_all.itertuples(index=True, name='Edge'):
    # Extract source, target, and other relevant columns from the row tuple
    # print(row[1])
    source = row[0][0]  # Extract u from the multi-index
    target = row[0][1]  # Extract v from the multi-index
    geometry = row[1]
    length = geometry.length
    
    # Append the edge to the edge list
    edge_list.append((source, target, {'geometry': geometry, 'length': length}))

# Display the edge list
# print(edge_list)

In [40]:
G = nx.from_edgelist(edge_list, create_using=nx.DiGraph())

### Shortest Path

In [148]:
path = nx.shortest_path(G, source=closest_edge_start[0], target=closest_edge_end[1], weight='length')

In [149]:
import geopandas as gpd
from shapely.geometry import LineString

# Initialize a list to store dictionaries of geometry
tmp_list = []

# Loop through the path vertices
for vertex in range(len(path)-1):
    u = path[vertex]
    v = path[vertex + 1]
    
    # Access the GeoSeries containing the geometry
    geometry_series = bikelane_all.loc[(u, v)]['geometry']
    
    # Check if the geometry series is not empty
    if not geometry_series.empty:
        # Access the LineString object within the GeoSeries
        geometry = geometry_series.iloc[0]
        
        # Check if the geometry is a LineString object
        if isinstance(geometry, LineString):
            # Append the LineString object to the list of dictionaries
            tmp_list.append({'geometry': geometry})
        else:
            # Handle unknown geometry types or skip them
            print(f"Skipped unknown geometry at index ({u}, {v})")
            continue
    else:
        # Handle empty geometry series
        print(f"No geometry found at index ({u}, {v})")
        continue

# Create LineString objects from the coordinates
line1 = LineString(coordinates1)
line2 = LineString(coordinates2)
line3 = LineString(coordinates3)

tmp_list.append({'geometry': line1})
tmp_list.append({'geometry': line2})
tmp_list.append({'geometry': line3})
# Create a GeoDataFrame from the list of dictionaries

geometries_gdf = gpd.GeoDataFrame(tmp_list)
geometries_gdf.crs = bikelane_all.crs

# Print the GeoDataFrame
geometries_gdf.explore()

C:\Users\Gergo Gyori\AppData\Local\Temp\ipykernel_26188\2423269188.py:13: PerformanceWarning: indexing past lexsort depth may impact performance.
  geometry_series = bikelane_all.loc[(u, v)]['geometry']


#### THIS IS IT

In [45]:
# Write to GraphML
nx.write_graphml(G_edges, 'graph.graphml')